In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import autokeras as ak
from tensorflow.keras.models import load_model

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import GroupShuffleSplit

In [2]:
params = {
    "test_size":0.33,
    "random_state":42,
    "splits":1,
    
}

In [3]:
df = pd.read_pickle("mid_concrete.pkl")

In [4]:
import os
os.getcwd()

'C:\\Users\\nini\\Desktop\\Libra\\sensors-positioning'

In [5]:
df.shape

(29268, 17)

In [6]:
df.dtypes

point                  int64
anchor                 int64
channel               object
x_tag                float64
y_tag                float64
true_theta           float64
true_phi             float64
power                float64
pdda_input_real_1    float64
pdda_input_real_2    float64
pdda_input_imag_2    float64
pdda_input_real_3    float64
pdda_input_imag_3    float64
pdda_input_real_4    float64
pdda_input_imag_4    float64
pdda_input_real_5    float64
pdda_input_imag_5    float64
dtype: object

In [7]:
df['channel'] = df['channel'].astype(str).astype(float)

In [8]:
# df.drop(['point', 'anchor', 'channel'], axis=1,inplace=True)

In [45]:
features = ['power', 'pdda_input_real_1', 'pdda_input_real_2', 'pdda_input_imag_2', 'pdda_input_real_3', 'pdda_input_imag_3', 'pdda_input_real_4', 'pdda_input_imag_4',
           'pdda_input_real_5', 'pdda_input_imag_5']
targets = ['true_phi']

In [46]:
gs = GroupShuffleSplit(n_splits=params["splits"], test_size=params["test_size"], random_state=params["random_state"])
train_idx, test_idx = next(gs.split(X, y, groups=df.point))

In [47]:
train = df.iloc[train_idx]
test = df.iloc[test_idx]
assert round(train.shape[0]/(train.shape[0]+test.shape[0]), 2) == 0.67
assert len(set(train.point) & set(test.point)) == 0

In [48]:
train.shape

(19614, 17)

In [49]:
train.head()

,point,anchor,channel,x_tag,y_tag,true_theta,true_phi,power,pdda_input_real_1,pdda_input_real_2,pdda_input_imag_2,pdda_input_real_3,pdda_input_imag_3,pdda_input_real_4,pdda_input_imag_4,pdda_input_real_5,pdda_input_imag_5
0,36,4,37.0,44.3,43.1,-78.7,-135.0,0.234891,0.607840,-0.043681,-0.623505,-0.687046,0.444869,0.539495,0.154464,0.374791,0.435915
1,36,4,38.0,44.3,43.1,-78.7,-135.0,0.241913,0.769467,-0.096220,-0.640024,-0.699148,0.398192,0.650191,0.195234,0.408465,0.452313
2,36,4,39.0,44.3,43.1,-78.7,-135.0,0.255959,0.726323,-0.436676,-0.479011,-0.484936,0.606550,0.731353,0.097643,0.614667,0.263012
3,35,3,37.0,44.1,49.9,-77.4,-108.4,0.104267,0.154127,-0.007433,-0.088215,-0.060157,0.121371,0.000587,0.152934,-0.070711,0.335963
4,35,3,38.0,44.1,49.9,-77.4,-108.4,0.123931,0.238340,-0.130614,-0.132987,-0.056824,0.197203,0.063315,0.106936,-0.053913,0.271928


In [180]:
train.to_csv('train.csv')

In [50]:
train.drop(['point', 'anchor', 'channel'], axis=1,inplace=True)

C:\Users\nini\AppData\Local\Temp\ipykernel_20628\1560788409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['point', 'anchor', 'channel'], axis=1,inplace=True)


In [56]:
X = df[features].values
y = df[targets].values

In [58]:
X_train = train[features].values
y_train = train[targets].values
X_test = test[features].values
y_test = test[targets].values

In [21]:
X_train.shape

(19614, 10)

In [186]:
X_train = np.expand_dims(
    X_train, axis=2
)  # This step it's very important an CNN will only accept this data shape

In [188]:
X_train.tolist()[0]

[[0.23489060198671224],
 [0.6078395947945477],
 [-0.04368084298452837],
 [-0.6235046787764795],
 [-0.6870458734119732],
 [0.4448685067832727],
 [0.5394947619541639],
 [0.15446367315333404],
 [0.3747913343437288],
 [0.43591494181875357]]

In [145]:
y_train.shape

(19614, 2)

In [146]:
input_node = ak.Input()
output_node = ak.Normalization()(input_node)

output_node = ak.ConvBlock()(output_node)
output_node = ak.SpatialReduction()(output_node)
output_node = ak.DenseBlock()(output_node)

output_node = ak.RegressionHead(loss="mean_absolute_error", metrics="mean_absolute_error")(output_node)

reg = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node, 
    max_trials=100, 
    overwrite=True,
    tuner='greedy'
)


In [147]:
reg.fit(X_train, y_train, batch_size=32, epochs=20)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |?                 |conv_block_1/kernel_size
False             |?                 |conv_block_1/separable
True              |?                 |conv_block_1/max_pooling
2                 |?                 |conv_block_1/num_blocks
2                 |?                 |conv_block_1/num_layers
32                |?                 |conv_block_1/filters_0_0
32                |?                 |conv_block_1/filters_0_1
0                 |?                 |conv_block_1/dropout
32                |?                 |conv_block_1/filters_1_0
32                |?                 |conv_block_1/filters_1_1
flatten           |?                 |spatial_reduction_1/reduction_type
False             |?                 |dense_block_1/use_batchnorm
2                 |?                 |dense_block_1/num_layers
32                |?                 |dense_block_1/units_0
0                 |?                

KeyboardInterrupt: 

In [51]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 10, 1)            0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 10, 1)            3         
 n)                                                              
                                                                 
 separable_conv1d (Separable  (None, 8, 32)            67        
 Conv1D)                                                         
                                                                 
 separable_conv1d_1 (Separab  (None, 6, 32)            1152      
 leConv1D)                                                   

In [20]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [21]:
# Predict with the best model.
predicted_y = model.predict(X_test)

302/302 [==============================] - 1s 3ms/step


In [22]:
# Evaluate the best model with testing data.
print(model.evaluate(X_test, y_test))

302/302 [==============================] - 1s 2ms/step - loss: 6.2333 - mean_squared_error: 124.6177
[6.233319282531738, 124.61772918701172]


In [23]:

print(type(model))

<class 'keras.engine.functional.Functional'>


In [24]:
try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    print(Exception)

INFO:tensorflow:Assets written to: model_autokeras\assets


INFO:tensorflow:Assets written to: model_autokeras\assets


In [165]:
loaded_model = load_model("model_autokeras")

In [166]:
print(loaded_model.evaluate(X_test, y_test))

302/302 [==============================] - 1s 3ms/step - loss: 4.6465 - mean_absolute_error: 4.6465
[4.64654016494751, 4.64654016494751]


In [167]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 10, 1)            0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 10, 1)            3         
 n)                                                              
                                                                 
 conv1d (Conv1D)             (None, 6, 32)             192       
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 256)            41216     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 256)           0     

In [168]:
for layer in loaded_model.layers:
    print(layer.get_output_at(0).get_shape().as_list())

[None, 10, 1]
[None, 10, 1]
[None, 10, 1]
[None, 6, 32]
[None, 6, 256]
[None, 2, 256]
[None, 2, 256]
[None, 2, 64]
[None, 1, 64]
[None, 64]
[None, 32]
[None, 32]
[None, 32]
[None, 256]
[None, 256]
[None, 256]
[None, 2]


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

In [37]:
X_train.shape

(19609, 10)

In [22]:
df = pd.read_pickle("mid_concrete.pkl")

In [23]:
df['channel'] = df['channel'].astype(str).astype(float)
features = ['power', 'pdda_input_real_1', 'pdda_input_real_2', 'pdda_input_imag_2', 'pdda_input_real_3', 'pdda_input_imag_3', 'pdda_input_real_4', 'pdda_input_imag_4',
           'pdda_input_real_5', 'pdda_input_imag_5']
# theta does not change, make predictions for phi only, azimuth angles 
# predict only for phi, 
targets = ['true_phi']
location = ['x_tag', 'y_tag']
X = df[features].values
y = df[targets].values
loc = df[location].values


In [32]:
accuracy, pipeline = train(model())
accuracy

ValueError: The first argument to `Layer.call` must always be passed.

In [80]:
def create_convnet(img_path='network_image.png'):
    input_shape = layers.Input(shape=(10,1))

    block_1 = layers.Conv1D(50, 10, padding='same', activation='relu')(input_shape)
    block_1 = layers.MaxPooling1D(1, padding='same')(block_1)

    block_2 = layers.Conv1D(50, 8, padding='same', activation='relu')(input_shape)
    block_2 = layers.MaxPooling1D(1, padding='same')(block_2)

    block_3 = layers.Conv1D(50, 6, padding='same', activation='relu')(input_shape)
    block_3 = layers.MaxPooling1D(1, padding='same')(block_3)
    
    block_4 = layers.Conv1D(50, 4, padding='same', strides=2, activation='relu')(input_shape)
    block_4 = layers.MaxPooling1D(1, padding='same')(block_4)
    
    block_5 = layers.Conv1D(50, 4, padding='same',  strides=4, activation='relu')(input_shape)
    block_5 = layers.MaxPooling1D(1, padding='same')(block_5)

    merged = layers.concatenate([block_1, block_2, block_3, block_4, block_5], axis=1)
    merged = layers.Flatten()(merged)
    
    # global average pooling to reduce filter dimension
    # compare this approach with 2 layers cnn, to check if this idea makes sense
    # check this model with edge impulse.
    # test only on points not seen in train -> channel
    # flatten across channel dimension
    # validate non shuffled samples
    # channels and samples should not overlap for training and testing
    # compare completely shuffled vs non overlapping apporoach
    
    # linear regression model on edge impulse, limited data 100 samples
    
    # thursday before lunch or from 1 to 2
    # explain what we did previous from old master's thesis
    # main goal -> embed Deep Learning model into small devices
    # positioning problem
    # architecture finding, deployement step
    # angle estimation, as well as generic approach
    # shrink model size with acceptable performance degradation
    # find good overla
    out = layers.Dense(1, activation='relu')(merged)
    model = keras.Model(input_shape, out)
    # keras.utils.plot_model(model, to_file=img_path)
    model.compile(loss='mean_absolute_error', optimizer='Adam')
    print(model.summary())
    return model

In [81]:
model = create_convnet()
model

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 10, 1)]      0           []                               
                                                                                                  
 conv1d_55 (Conv1D)             (None, 10, 50)       550         ['input_12[0][0]']               
                                                                                                  
 conv1d_56 (Conv1D)             (None, 10, 50)       450         ['input_12[0][0]']               
                                                                                                  
 conv1d_57 (Conv1D)             (None, 10, 50)       350         ['input_12[0][0]']               
                                                                                           

In [78]:
# add another dense layer before the output layer
# use mean squared error 
# plot plane of how the errors are distributed
# use batch norm
# use data augmentation, add random noise check libra paper for their 
# calculate r^2 for the simple models
# calculate  's and pearson's cooeficient
# timonmerk github username for Timon
# include ppda_azimuth in dataframe and upload on github repo
# prepare powerpoint ppts for meetings
# hyperparameter tuning for simple models
# plot pdda box plot 
# also check the  of scikit learn models
# cnn model check how it can be improved. 
# plot the features look out for outliers, how is the data distributed, before i normalize the data
# after the first cnn layer do the batchnorm
# prepare the gantt chart
# 
X_train = np.expand_dims(
    X_train, axis=2
)  # This step it's very important an CNN will only accept this data shape

In [84]:
model.fit(X_train,y_train, batch_size=10, epochs=10)

Epoch 1/10
1962/1962 [==============================] - 7s 3ms/step - loss: 17.2111
Epoch 2/10
1962/1962 [==============================] - 7s 4ms/step - loss: 17.2031
Epoch 3/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.2050
Epoch 4/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.2026
Epoch 5/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.1853
Epoch 6/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.1740
Epoch 7/10
1962/1962 [==============================] - 7s 3ms/step - loss: 17.1756
Epoch 8/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.1659
Epoch 9/10
1962/1962 [==============================] - 6s 3ms/step - loss: 17.1649
Epoch 10/10
1962/1962 [==============================] - 7s 3ms/step - loss: 17.1560
